<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/23Module_4_Foundation_semantic_search_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# With this operation, a comprehensive language model is downloaded from the 'spacy' library. There are three versions to it:
# - Large
# - Medium
# - Small
# In this case, we are using the smaller model, which is around 30 MB.

!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
!pip install -U sentence-transformers rank_bm25

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Read the data for Paris Hotels

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/paris_02_11_23.csv')

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [ ]:
df.shape

(28694, 18)

In [ ]:
# Count the number of times a specific name appears in the 'name' column, and relay that number for each different name, in order.
df.name.value_counts()

,count
name,
"Hotel d'Angleterre, Saint Germain des Pres",960
InterContinental Paris - Le Grand,800
Hotel Marceau Champs Elysees,800
Sonder Atala Champs-Élysées,800
Hotel Dauphine Saint Germain,800
...,...
Hotel B55,94
Les Jardins du Faubourg,78
Ibis Styles Paris Meteor Avenue de la Porte d'Italie,62


In [ ]:
# Drop the duplicated values from each column, i.e. drop a row if it contains a duplicated value.

df=df.drop_duplicates()

In [ ]:
df.shape

(11990, 18)

Create a new folder in Google Drive - called "Semantic_Search"

In [ ]:
df.name.value_counts()

,count
name,
InterContinental Paris - Le Grand,80
Okko Hotels Paris Porte de Versailles,80
Citadines Montmartre Paris,80
Hotel Montfleuri,80
Hotel Cayre,80
...,...
Seven Hotel Paris,32
Ibis Styles Paris Meteor Avenue de la Porte d'Italie,31
Hotel Du Sentier,28


In [ ]:
# Make a folder in your drive folder called "Semantic_Search".
!mkdir /content/drive/MyDrive/Semantic_Search

mkdir: cannot create directory ‘/content/drive/MyDrive/Semantic_Search’: File exists


In [ ]:
# Save the dataframe to the folder you just creagted.
df.to_csv('/content/drive/MyDrive/Semantic_Search/paris_02_11_23.csv',index=False)

In [ ]:
df.shape

(11990, 18)

In [ ]:
df.name.value_counts()

,count
name,
InterContinental Paris - Le Grand,80
Okko Hotels Paris Porte de Versailles,80
Citadines Montmartre Paris,80
Hotel Montfleuri,80
Hotel Cayre,80
...,...
Seven Hotel Paris,32
Ibis Styles Paris Meteor Avenue de la Porte d'Italie,31
Hotel Du Sentier,28


In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [ ]:
# Create a column named 'combined', which containes the titles of the different lodges, with the descriptions associated to it.
df["combined"] = (
    "title: " + df.title.str.strip()+"; Content: " + df.text.str.strip()
    # +"; desc: "+ df.text.str.strip()
)

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: A large impersonal place with an on tim...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Good hotel with rude waiter; Content: W...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Fantastic; Content: Absolutely top-notc...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...","title: Amidst the chaos of Fashion week, their..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Not worth the effort or money; Content:...


In [ ]:
import re

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

# Translate all the "combined" column to lower case.
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))


In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

GPU Found!


In [ ]:
df_combined.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...


In [ ]:
# Take a sample of the first 10 rows.
sample = df_combined[:10]

In [ ]:
sample

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,/ShowUserReviews-g187147-d207742-r862073785-In...,en,MOBILE,BA9F96090F35C7A562E550825BAB4B32,badgerken2019,badgerken2019,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not quite up to intercontinental standar...
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,/ShowUserReviews-g187147-d207742-r861021031-In...,en,OTHER,CB65D8DF04EEB78702BB03F39BAC6717,BadBenito,BadBenito,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title luxury in the heart of the city content ...
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,/ShowUserReviews-g187147-d207742-r860421664-In...,en,OTHER,9D8328D86D62AFA560367B6F81565010,Shivers2612,Shivers2612,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title beautiful content visited for my birthda...
8,856124006,2022-08-24,5,Great old fashioned quality service,All I can say is we found good old fashioned f...,1,/ShowUserReviews-g187147-d207742-r856124006-In...,en,OTHER,82C9562F2314BE52F0CD417EC6034CE2,Nouf M,noufm716,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title great old fashioned qualit

In [ ]:
# Switch to CPU
embedder =  embedder.to('cpu')

startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 5.193675756454468


<ipython-input-25-8160b5948b46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:
# Switch once more to GPU.

embedder =  embedder.to('cuda')
startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# Notice the difference in time to do this operation between GPU and CPU.


Execution time in seconds: 1.5070955753326416


<ipython-input-26-c2ada10f2885>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:
sample

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.01282531, 0.0029111477, 0.070..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.05525053, 0.019289287, -0.0056739757, 0.057..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940896, 0.022968661, -0.0036183984, 0.0..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499605, 0.04220707, -0.006234068, 0.06..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013607198, -0.0038351824, 0.0018077933, 0...."
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,/ShowUserReviews-g187147-d207742-r862073785-In...,en,MOBILE,BA9F96090F35C7A562E550825BAB4B32,badgerken2019,badgerken2019,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not quite up to intercontinental standar...,"[0.041726097, 0.033815235, 0.0017297651, 0.065..."
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,/ShowUserReviews-g187147-d207742-r861021031-In...,en,OTHER,CB65D8DF04EEB78702BB03F39BAC6717,BadBenito,BadBenito,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title luxury in the heart of the city content ...,"[-0.0469322, -0.017643088, 0.024403226, 0.0763..."
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,/ShowUserReviews-g187147-d207742-r860421664-In...,en,OTHER,9D8328D86D62AFA560367B6F81565010,Shivers2612,Shivers2612,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title beautiful content visited for my birthda...,"[0.007762895, 0.033725765, 0.00603

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# Transform your dataframe to a pickle file, which is a byte stream file used to save a dataframe's state across sections.
sample.to_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl')    #to save the dataframe, df to 123.pkl

In [ ]:
# Load the pickle file.
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df_with_embedding.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.01282531, 0.0029111477, 0.070..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.05525053, 0.019289287, -0.0056739757, 0.057..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940896, 0.022968661, -0.0036183984, 0.0..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499605, 0.04220707, -0.006234068, 0.06..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013607198, -0.0038351824, 0.0018077933, 0...."


In [ ]:
query = 'Not worth the effort or money + This hotel is not worth the effort or the price'

# Embed the previous query.
query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# First element of the 'combined' column.
df_with_embedding.combined[0]

'title a large impersonal place with an on time check in problem and other issues content if you are looking for a huge grand hotel experience this place may be for you but i found it to be impersonal and the staff lacking warmth and sometimes manners they seem to avoid engagement with guests whenever possible add that to the fact that each morning and evening there are 2 or 3 bus loads of tour groups gathering in the lobby and right outside the hotel when i encountered this on my first evening and needed a taxi i asked the doorman whether he could get me a taxi or whether i should order an uber he simply nodded and walked away so i ordered an uber\n\nthe checkin counter must be understaffed as i had to wait in a long line for about 15 minutes only to be told that my room was not ready official checkin is 2pm but my room was not ready until 5pm i should have taken seriously the many previous tripadvisor reviewers who had the same experience they gave me a drink voucher in the bar as co

In [ ]:
# Create a list of embeddings, from the contents of the "combined" column.
corpus_embeddings = embedder.encode(df_with_embedding.combined,show_progress_bar=True)

# Create a column called 'similarity', displaying the cosine similarity between your embedded query and each embedded content from the 'corpus_embeddings' variable.
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_with_embedding

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,author_username,name,id,description,rating,rating_count,features,combined,embedding,similarity
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,...,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.01282531, 0.0029111477, 0.070...",0.180471
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,...,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.05525053, 0.019289287, -0.0056739757, 0.057...",0.144596
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,...,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940896, 0.022968661, -0.0036183984, 0.0...",0.154253
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,...,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499605, 0.04220707, -0.006234068, 0.06...",0.151105
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,...,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013607198, -0.0038351824, 0.0018077933, 0....",0.255166
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,/ShowUserReviews-g187147-d207742-r862073785-In...,en,MOBILE,BA9F96090F35C7A562E550825BAB4B32,...,badgerken2019,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not quite up to intercontinental standar...,"[0.041726097, 0.033815235, 0.0017297651, 0.065...",0.151502
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,/ShowUserReviews-g187147-d207742-r861021031-In...,en,OTHER,CB65D8DF04EEB78702BB03F39BAC6717,...,BadBenito,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title luxury in the heart of the city content ...,"[-0.0469322, -0.017643088, 0.024403226, 0.0763...",0.105180
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,/ShowUserReviews-g187147-d207742-r860421664-In...,en,OTHER,9D8328D86D62AFA560367B6F81565010,...,Shivers2612,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title beautiful content visited for my birthda...,"[0.007762895, 0.

In [ ]:
# cosine_similarity(corpus_embeddings, query_embedding)

#Entire Data

In [ ]:
startTime = time.time()

# Create a column named 'embedding', where the 'combined' column is turned to embeddings by the model.
df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 194.0840504169464


In [ ]:
# Turn the dataframe you have just created to pickle file, for later use.
df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

##Embedding upload - new starting point


In [ ]:
!pip install openai==0.27.7
!pip install gradio
!pip install -U sentence-transformers

In [ ]:
# drive.mount() loads the contents from your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

embedder = SentenceTransformer('all-mpnet-base-v2')

# Use the GPU if available
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

GPU Found!


In [ ]:
# Read the pickle file you entered earlier.
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df.shape

(11990, 20)

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.01282531, 0.0029111477, 0.070..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.05525053, 0.019289287, -0.0056739757, 0.057..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940896, 0.022968661, -0.0036183984, 0.0..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499605, 0.04220707, -0.006234068, 0.06..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013607198, -0.0038351824, 0.0018077933, 0...."


In [ ]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):

    # Embed your search query.
    query_embedding = embedder.encode(query,show_progress_bar=True)

    # As before, create a 'similarity' column, which shows the cosine similarity between the your query and the embedded combined contents.
    # REMEMBER!! YOU ARE USING THE SAME MODEL TO EMBED BOTH THE COMBINED CONTENTS AND YOUR QUERY.
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1))) #similarity against each doc

    # Now, sort the values bty similarity, and choose the most similar doc.
    results = (
        df.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [ ]:
query = 'close to Louvre and great food nearby but not too expensive'

In [ ]:
results = search_reviews(df,query,15)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,author_username,name,id,description,rating,rating_count,features,combined,embedding,similarity
15436,698261802,2019-08-12,5,Recommend!,Just minutes walk from the Louvre Museums and ...,2,/ShowUserReviews-g187147-d197426-r698261802-Ci...,en,MOBILE,AA83C9E78CC6EC43EDC559DC7C0CDF57,...,Loverofgoodfood1969,Citadines Les Halles Paris,197426,Apart'hotel Citadines Les Halles welcomes you ...,4.0,1426.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title recommend content just minutes walk from...,"[-0.02156358, 0.022654237, 0.003227969, 0.0615...",[0.7623061]
10033,858737654,2022-09-06,4,Great hotel close to the Louvre,This was a great location with easy access to ...,1,/ShowUserReviews-g187147-d228694-r858737654-Ho...,en,MOBILE,107486993911F044C7A86C88BD8CBE6C,...,kahnfeldt,Hotel Malte - Astotel,228694,Located in the 2nd district next to the Stock ...,5.0,2285.0,"['roomFeatures_allergy-free room', 'roomFeatur...",title great hotel close to the louvre content ...,"[0.00507981, 0.006933634, 0.0052260924, 0.0466...",[0.7389944]
14024,497037862,2017-06-28,5,Fantastic Location and Service- Highly Recomme...,"Best Location in Paris, 3 minutes walking dist...",0,/ShowUserReviews-g187147-d197966-r497037862-Ho...,en,OTHER,5271CD2C9F388D5BCE79D0F5598CC4FF,...,Modhura,Hotel Folkestone Opera,197966,"Welcome to the Folkestone Opera Hotel, your Pa...",4.5,870.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title fantastic location and service highly re...,"[0.005145668, 0.033328705, 0.00956577, 0.06224...",[0.7292295]
28275,801368976,2021-08-02,5,"Best location, elegant, clean and great rooms.",We spent 5 nights at the lovely Hotel Da Vinci...,1,/ShowUserReviews-g187147-d6675948-r801368976-H...,en,MOBILE,1FEA86D03026BB83FEB43E29EA2B2D0D,...,marydresser,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN,title best location elegant clean and great ro...,"[-0.04500474, 0.030609895, 0.0011573076, 0.069...",[0.7261323]
16159,727252844,2019-11-18,5,Perfect spot to explore Paris,This was our first visit to Paris and stumbled...,1,/ShowUserReviews-g187147-d635192-r727252844-Re...,en,OTHER,EDB7ED82C206CE0E8692D212CB2B4A4A,...,AlexanderMeerkat,Residence & Spa Le Prince Regent,635192,The Residence & Spa le Prince located in Saint...,4.5,471.0,"['roomFeatures_air conditioning', 'roomFeature...",title perfect spot to explore paris content th...,"[-0.0394307, 0.045125075, 0.009108177, 0.04777...",[0.7245367]
6866,806790088,2021-08-28,5,Nice small hotel at an excellent location,"Clean, with a nice design and an excellent loc...",0,/ShowUserReviews-g187147-d278169-r806790088-Ho...,en,OTHER,3C6E94BBBCCA399CFE39851FEF397129,...,do8yb,Hôtel Eugène en Ville,278169,NaN,NaN,NaN,NaN,title nice small hotel at an excellent locatio...,"[-0.013086394, -0.02277145, 0.021511065, 0.072...",[0.71964693]
207,834926510,2022-04-17,5,Best Paris Location,You can’t beat the location of this hotel! Rig...,0,/ShowUserReviews-g187147-d228728-r834926510-Re...,en,MOBILE,58EB5EA5E1A2FCD598F7F7F9D988306A,...,Camper46227911888,Renaissance Paris Vendome Hotel,228728,Bask in the lavish lifestyle of our boutique h...,4.5,1618.0,"['roomFeatures_bathrobes', 'roomFeatures_air c...",title best paris location content you cant bea...,"[-0.0070767864, -0.023154905, 0.011793152, 0.0...",[0.718215]
6233,227310602,2014-09-06,5,"Fantastic place, good value and well connected","A real find; it's comfortable, quiet, and welc...",1,/ShowUserReviews-g187147-d613301-r227310602-A_...,en,MOBILE,466D8F84569A260E4A3F39A32DDB3AC7,...,bertiebr,A Room In Paris,613301,Right in the city centre of Paris 5 beautiful ...,4.5,237.0,"['roomFeatures_seating area', 'roomFeatures_ir...",title fantastic place good value and well conn...,"[-0.024605073, 0.005065016, 0.024327451, 0.060...",[0.7173573]
4933,815954462,2021-10-25,5,Great location for romantic trip,"Friendly staff. Great location, relaxed atmosp...",0,/ShowUserReviews-g187147-d228737-

Take all the reviews which are closest to the query, and groupby the hotel name

In [ ]:
def search(query):
  # Define a number of results to return, in this case, return only the first 15 results ranked by similarity.
  n = 15

  # Embed the query.
  query_embedding = embedder.encode(query)

  # Generate the similarity column, based on your query.
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  # Calculate the top 'n' most similar results by similarity.
  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  # Display them in a very concise and ordered manner.
  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist




In [ ]:
search('close to Louvre and great food nearby but not too expensive')

[{'name': 'Citadines Les Halles Paris',
  'score': 0.7623061,
  'rating': 4.0,
  'relevant_reviews': ['Just minutes walk from the Louvre Museums and steps to all levels of dining establishments. Could not have been happier when we opened the door to our base rate room to find we had French doors opening to a little balcony with views to die for. It’s a busy city so if you want to be central there plenty of action outside but once the door was closed there was very little noise. Great to have a refrigerator/sink/cutlery/crockery/glassware. Really clean and modern place, fabulous find for the price bracket. Hotel has a laundry for guest use which is handy. Rooms are not serviced, which we liked - no need to prepare your personal items before staff arrive. Second visit and not my last. Staff at check in were excellent/English speaking and we purchased hop on hop off bus tickets there at a reduced price. ']},
 {'name': 'Hotel Malte - Astotel',
  'score': 0.7389944,
  'rating': 5.0,
  'rele

In [ ]:
from google.colab import userdata
import openai
openai.api_key = userdata.get('OPENAI_API_KEY')


In [ ]:
def generate_answer(query):
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context
    focusing on which is the top hotel based on the query. You should responsd as you a travel agent and are conversing with the
    user

    ###########
    query:
    "{query}"

    ########

    context:"
    "{search(query)}"
    #####

    Return in Markdown format with each hotel highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.7,
        messages = messages


    )

    # Extract the generated response from the API response
    generated_text = response.choices[0].message['content'].strip()

    return generated_text

# # Example usage
# query = "What are the best amenities offered by Hotel XYZ?"
# response = generate_hotel_response(query)
# print(response)

    return response.choices[0].message.content.strip()

In [ ]:
txt=generate_answer('close to Louvre and great food nearby but not too expensive')

In [ ]:
import markdown
from IPython.display import display, HTML

def render_markdown(md_text):
    # Convert Markdown to HTML
    html = markdown.markdown(md_text)
    # Display the HTML
    display(HTML(html))

In [ ]:
render_markdown(txt)

#Building the API

In [ ]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          sm =smalldf.rating[r].mean()

          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "description":results.description[r],
            "relevance score": smallarr.tolist(),
            "rating": sm.tolist(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

def greet(query):
    bm25 = search(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="json")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)

# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3218dd1e1dc4582153.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --upgrade gradio

In [ ]:
# Import the Client class from the gradio_client module.
from gradio_client import Client

# Create an instance of the Client class. The URL provided should point to a live Gradio app.

# PASTE THE URL FROM ABOVE HERE
client = Client("https://3218dd1e1dc4582153.gradio.live")

# Use the 'predict' method of the Client instance to send a request to the Gradio app.
# The string "Hotel near the Eiffel Tower!" is passed to the 'query' textbox component of the app.
# 'api_name' specifies the endpoint ('/predict') that the Gradio interface exposes for processing this input.
result = client.predict(
				"Hotel near the Eiffel Tower!",	# str in 'query' Textbox component
				api_name="/predict"
)

# Print the result returned from the Gradio app. This output depends on how the Gradio app processes the input.
print(result)



Loaded as API: https://3218dd1e1dc4582153.gradio.live/ ✔
[{'name': 'Le Parisis Paris Tour Eiffel', 'description': None, 'relevance score': 0.8320505619049072, 'rating': 4.5, 'relevant_reviews': ['Really lovely decor, hotel room was clean, hotel staff were friendly and had great customer service, amazing location literally 10 min walk from the Eiffel tower and in between two metro stations. Would recommend!']}, {'name': 'Hotel Duquesne Eiffel', 'description': 'This cosy hotel offers numerous advantages such as its geographic location, appreciated as much for "leisure" vacations as it is for business meetings. Between "les Invalides" and the Eiffel Tower, the Duquesne Eiffel Hotel is located in one of Paris\' most pleasant boroughs on the famous left bank of the Seine. The team welcomes you with a smile 24/7, while taking all the necessary measures to ensure that your stay is a pleasant one.', 'relevance score': 0.8285293579101562, 'rating': 4.5, 'relevant_reviews': ['The location of thi

In [ ]:
result

[{'name': 'Le Parisis Paris Tour Eiffel',
  'description': None,
  'relevance score': 0.8320505619049072,
  'rating': 4.5,
  'relevant_reviews': ['Really lovely decor, hotel room was clean, hotel staff were friendly and had great customer service, amazing location literally 10 min walk from the Eiffel tower and in between two metro stations. Would recommend!']},
 {'name': 'Hotel Duquesne Eiffel',
  'description': 'This cosy hotel offers numerous advantages such as its geographic location, appreciated as much for "leisure" vacations as it is for business meetings. Between "les Invalides" and the Eiffel Tower, the Duquesne Eiffel Hotel is located in one of Paris\' most pleasant boroughs on the famous left bank of the Seine. The team welcomes you with a smile 24/7, while taking all the necessary measures to ensure that your stay is a pleasant one.',
  'relevance score': 0.8285293579101562,
  'rating': 4.5,
  'relevant_reviews': ['The location of this hotel couldn\'t be beat. We had the to

In [ ]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          sm =smalldf.rating[r].mean()

          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "description":results.description[r],
            "relevance score": smallarr.tolist(),
            "rating": sm.tolist(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

def generate_answer(query):
    bm25 = search(query)
    return bm25

# Use the gradio library to display a user interface for your user to interact with.
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

# Launch the user demo - This can be done directly in your colab notebook. On your local notebook, you can also give a personalized localhost:port address.
demo.launch(share=True,debug=False)

# COPY THE URL THAT APPEARS BELOW

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5c757f500639ae2b17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
